## Import libraries


In [3]:
import pandas as pd
import os
from tqdm import tqdm
from joblib import Parallel, delayed

WORK_DIR = "/beegfs/halder/GITHUB/RESEARCH/crop-yield-forecasting-germany/"
os.chdir(WORK_DIR)

In [2]:
CROP = "winter_wheat"

## Define the paths


In [6]:
data_dir = os.path.join(WORK_DIR, "interim")
climate_dir = os.path.join(data_dir, "climate", CROP)
rs_dir = os.path.join(data_dir, "remote_sensing", CROP)
out_dir = os.path.join(WORK_DIR, "data", "processed", CROP, "timeseries_7D")

os.makedirs(out_dir, exist_ok=True)